In [1]:
import os
os.chdir('../')

In [2]:
%pwd

'd:\\pythonweb\\Image_classification\\Kidney-disease-classification-mlflow-'

In [3]:
import mlflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="http://127.0.0.1:8080",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [9]:
import setuptools
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                 mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model", )
                 #mlflow.tensorflow.load_model(self.model, "model", registered_model_name="VGG16Model", )
            else:
                 mlflow.keras.log_model(self.model, "model")
                # mlflow.tensorflow.load_model(self.model, "model", )

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
except Exception as e:
   raise e

[2024-11-30 12:13:40,487: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-11-30 12:13:40,496: INFO: common: yaml file: params.yaml loaded successfully]
[2024-11-30 12:13:40,498: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 13s 1s/step - loss: 7.6741 - accuracy: 0.5036
[2024-11-30 12:13:53,874: INFO: common: json file saved at: scores.json]


2024/11/30 12:13:53 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-11-30 12:13:54,854: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\tv078\AppData\Local\Temp\tmp25_ox3ud\model\data\model\assets
[2024-11-30 12:13:55,407: INFO: builder_impl: Assets written to: C:\Users\tv078\AppData\Local\Temp\tmp25_ox3ud\model\data\model\assets]


2024/11/30 12:14:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/11/30 12:14:01 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
2024/11/30 12:14:01 INFO mlflow.tracking._tracking_service.client: 🏃 View run loud-crab-2 at: http://127.0.0.1:8080/#/experiments/0/runs/9dfd85a09afb46a6a89bd1b171bd1a43.
2024/11/30 12:14:01 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:8080/#/experiments/0.


In [19]:




from dataclasses import dataclass
@dataclass
class Table:
    ID : int
    Name : str
    Role : str
    Deg : str
    Depo : int









In [22]:
class comp:
    def __init__(self):
        pass
    def get_the_conf(self) -> Table:
        setvalues = Table(
        ID = 1,
        Name = 'Viswa',
        Depo = "4",
        Role = 'ST',
        Deg = "DS",

        )
        return setvalues
        

In [30]:
class showx:
    def __init__(self, configaration : comp):
        self.conf=configaration

    def printx(self):
        print(self.conf.ID, end="-->")
        print(self.conf.Name)

In [31]:
showx(comp().get_the_conf()).printx()

1-->Viswa
